In [5]:
BASE_URL = "https://goodinfo.tw/tw/StockDetail.asp?STOCK_ID=2330"

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def get_basic_info_selenium(stock_id):
    url = f"https://goodinfo.tw/tw/BasicInfo.asp?STOCK_ID={stock_id}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # 不開啟視窗
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)  # 等待 JS 載入真正內容

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    # 解析基本資料
    info = {}
    rows = soup.find_all("tr")
    for row in rows:
        ths = row.find_all("th", class_="bg_h1")
        tds = row.find_all("td", bgcolor="white")
        for th, td in zip(ths, tds):
            key = th.get_text(strip=True).replace("\u3000", "")
            val = td.get_text(strip=True)
            if key and val:
                info[key] = val

    return info

info = get_basic_info_selenium("2330")
for k, v in info.items():
    print(f"{k}：{v}")

股票代號：2330
股票名稱：台積電
產業別：半導體業
上市/上櫃：上市
公司名稱：台灣積體電路製造股份有限公司
英文簡稱：TSMC
成立日期：1987/02/21(38.3年)
掛牌日期：1994/09/05(30.8年)
上市日期：1994/09/05
上櫃日期：-
興櫃日期：-
公開發行日期：1992/10/30
資本額：2,593.26億元
每股面值：新台幣 10元
目前市值：25.8兆
公司債發行：有
發行股數：25,932,615,521股(含私募0股)
特別股：0股
財報編製類型：合併
盈餘分派頻率：每季
董事長：魏哲家
總經理：總裁: 魏哲家
發言人：黃仁昭
發言人職稱：資深副總經理暨財務長
代理發言人：高孟華
統一編號：22099131
總機電話：03-5636688
傳真電話：03-5797337
公司電郵：invest@tsmc.com
公司網址：https://www.tsmc.com
中文地址：新竹科學園區力行六路8號
英文地址：No. 8, Li-Hsin Rd. 6, Hsinchu Science Park,
Hsin-Chu 300096, Taiwan, R.O.C.
主要業務：依客戶之訂單與其提供之產品設計說明，以從事製造與銷售積體電路以及其他晶圓半導體裝置。提供前述產品之封裝與測試服務、積體電路之電腦輔助設計技術服務。提供製造光罩及其設計服務。
股票過戶機構：中國信託商業銀行 代理部
過戶機構電話：02-6636-5566
過戶機構地址：台北市重慶南路一段83號5樓
簽證會計師事務所：勤業眾信聯合會計師事務所
簽證會計師：吳世宗, 林尚志
投資人關係聯絡人：蘇志凱
聯絡人職稱：處長
投資人關係聯絡電話：03-563-6688
聯絡人電子郵件：jeff_su@tsmc.com
利害關係人專區網址：https://esg.tsmc.com/zh-Hant/sustainable-management/materiality-analysis


In [7]:
import re
from datetime import datetime

# 中文欄位對應到標準英文欄位名稱
COLUMN_MAP = {
    "股票代號": "stock_id",
    "股票名稱": "stock_name",
    "公司名稱": "company_name",
    "英文簡稱": "company_name_en",
    "成立日期": "established_date",
    "掛牌日期": "listing_date",
    "資本額": "capital_amount",
    "公司債發行": "has_corporate_bond",
    "發行股數": "issued_shares",
    "公司電話": "tel",
    "公司網址": "website",
    "董事長": "chairman",
    "總經理": "general_manager",
    "發言人": "spokesperson",
    "代理發言人": "deputy_spokesperson",
    "主要業務": "business_scope",
    "產業別": "industry",
    "統一編號": "tax_id",
}

# 將中文日期轉為 ISO 格式
def normalize_date(text):
    try:
        return datetime.strptime(text, "%Y/%m/%d").strftime("%Y-%m-%d")
    except:
        return None

# 將帶有單位的數字轉為 float 或 int
def clean_numeric(text):
    if "億" in text:
        num = re.sub(r"[^\d.]", "", text)
        return int(float(num) * 1e8)
    elif "萬" in text:
        num = re.sub(r"[^\d.]", "", text)
        return int(float(num) * 1e4)
    else:
        # 直接移除逗號、股等文字
        return int(re.sub(r"[^\d]", "", text)) if re.search(r"\d", text) else None

# 清洗 info 字典
def clean_basic_info(raw_info):
    cleaned = {}
    for zh_key, value in raw_info.items():
        en_key = COLUMN_MAP.get(zh_key, None)
        if not en_key:
            continue  # 忽略不在對照表內的欄位

        # 日期欄位轉換
        if "日期" in zh_key:
            cleaned[en_key] = normalize_date(value)
        # 數值欄位轉換
        elif re.search(r"(資本額|股數|股|億|萬)", zh_key):
            cleaned[en_key] = clean_numeric(value)
        # 其他文字欄位
        else:
            cleaned[en_key] = value.strip()

    return cleaned

# 使用範例
raw_info = get_basic_info_selenium("2330")
cleaned_info = clean_basic_info(raw_info)

# 輸出結果
for k, v in cleaned_info.items():
    print(f"{k}: {v}")


stock_id: 2330
stock_name: None
industry: 半導體業
company_name: 台灣積體電路製造股份有限公司
company_name_en: TSMC
established_date: None
listing_date: None
capital_amount: 259326000000
has_corporate_bond: 有
issued_shares: 259326155210
chairman: 魏哲家
general_manager: 總裁: 魏哲家
spokesperson: 黃仁昭
deputy_spokesperson: 高孟華
tax_id: 22099131
website: https://www.tsmc.com
business_scope: 依客戶之訂單與其提供之產品設計說明，以從事製造與銷售積體電路以及其他晶圓半導體裝置。提供前述產品之封裝與測試服務、積體電路之電腦輔助設計技術服務。提供製造光罩及其設計服務。


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_full_kline_data(stock_id="2330"):
    url = f"https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={stock_id}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    table = soup.find("table", id="tblDetail")
    if not table:
        raise Exception("找不到 tblDetail 表格 ─ Goodinfo 可能改版了")

    rows = table.find_all("tr")
    data = []
    for row in rows:
        cols = row.find_all("td")
        values = [td.text.strip().replace(",", "").replace("\xa0", "") for td in cols if td.text.strip()]
        if len(values) >= 20:  # 避免標題列或空列
            data.append(values)

    # 建立 DataFrame，欄位名稱參考截圖
    columns = [
        "日期", "開盤", "最高", "最低", "收盤", "漲跌", "漲跌幅", "振幅(%)",
        "張數", "筆數", "均價", "億元",
        "外資", "投信", "自營", "合計", "外資持股(%)",
        "融資增減", "融資餘額", "融券增減", "融券餘額", "券資比(%)"
    ]
    df = pd.DataFrame(data, columns=columns[:len(data[0])])  # 避免欄位數對不上

    return df

# 使用示範
df = get_full_kline_data("2330")
print(df.head())


          日期    開盤    最高   最低   收盤   漲跌    漲跌幅 振幅(%)     張數      筆數  ...  \
0  '25/06/06   997   997  991  995   -3   -0.3   0.6  18797   27283  ...   
1  '25/06/05  1000  1000  991  998   +8  +0.81  0.91  27048   51309  ...   
2  '25/06/04   974   990  970  990  +40  +4.21  2.11  43196   73539  ...   
3  '25/06/03   960   965  950  950   +4  +0.42  1.59  27483   43550  ...   
4  '25/06/02   958   961  946  946  -21  -2.17  1.55  40608  125245  ...   

       外資     投信    自營      合計 外資持股(%)   融資增減   融資餘額  融券增減 融券餘額 券資比(%)  
0   +1004   +238  +165   +1407    72.6   -149  19806  -384   12   0.06  
1   +9823   +689  -725   +9788    72.7   -424  19955  -167  396   1.98  
2  +15827   +479  +462  +16768    72.7  -1185  20379  -181  563   2.76  
3    -656   +146  -354    -863    72.6   -148  21564    -2  744   3.45  
4  -13196  +23.6  -586  -13759    72.6   +314  21712   -51  746   3.44  

[5 rows x 22 columns]


In [10]:
import re
import pandas as pd
from datetime import datetime

def convert_date_minguo(date_str):
    """將民國年日期轉為 ISO 格式"""
    try:
        if re.match(r"^\d{2}/\d{2}/\d{2}$", date_str.strip("'")):
            year, month, day = map(int, date_str.strip("'").split("/"))
            year += 1911  # 民國轉西元
            return f"{year:04d}-{month:02d}-{day:02d}"
    except:
        return None
    return None

def clean_numeric(val):
    """處理帶符號、千分位、空格的數值欄位"""
    if val in ["", "-", "—"]:
        return None
    val = val.replace(",", "").replace("+", "").replace("%", "")
    try:
        return float(val)
    except:
        return None

def clean_kline_dataframe(df):
    df = df.copy()

    # 日期欄位轉換
    if "日期" in df.columns:
        df["日期"] = df["日期"].apply(convert_date_minguo)

    # 欄位標準化（轉 float 的欄位）
    float_cols = [
        "開盤", "最高", "最低", "收盤", "漲跌", "漲跌幅", "振幅(%)",
        "張數", "筆數", "均價", "億元",
        "外資", "投信", "自營", "合計", "外資持股(%)",
        "融資增減", "融資餘額", "融券增減", "融券餘額", "券資比(%)"
    ]
    for col in float_cols:
        if col in df.columns:
            df[col] = df[col].apply(clean_numeric)

    return df

df = get_full_kline_data("2330")
cleaned_df = clean_kline_dataframe(df)
print(cleaned_df.head())

           日期      開盤      最高     最低     收盤    漲跌   漲跌幅  振幅(%)       張數  \
0  1936-06-06   997.0   997.0  991.0  995.0  -3.0 -0.30   0.60  18797.0   
1  1936-06-05  1000.0  1000.0  991.0  998.0   8.0  0.81   0.91  27048.0   
2  1936-06-04   974.0   990.0  970.0  990.0  40.0  4.21   2.11  43196.0   
3  1936-06-03   960.0   965.0  950.0  950.0   4.0  0.42   1.59  27483.0   
4  1936-06-02   958.0   961.0  946.0  946.0 -21.0 -2.17   1.55  40608.0   

         筆數  ...       外資     投信     自營       合計  外資持股(%)    融資增減     融資餘額  \
0   27283.0  ...   1004.0  238.0  165.0   1407.0     72.6  -149.0  19806.0   
1   51309.0  ...   9823.0  689.0 -725.0   9788.0     72.7  -424.0  19955.0   
2   73539.0  ...  15827.0  479.0  462.0  16768.0     72.7 -1185.0  20379.0   
3   43550.0  ...   -656.0  146.0 -354.0   -863.0     72.6  -148.0  21564.0   
4  125245.0  ... -13196.0   23.6 -586.0 -13759.0     72.6   314.0  21712.0   

    融券增減   融券餘額  券資比(%)  
0 -384.0   12.0    0.06  
1 -167.0  396.0    1.98  
2 